# Analytical Dataset Exploration

## Date Created: 1/20/22
## Date Modified: 3/7/22 (Millie)

### Authors: Geri, Millie, JJ

This notebook serves as a place to do data exploration on the `initial_analytic_dataset.csv` which holds all the variables of interest from each of our respective datasets. This is a place to look at the relationships between variables and other assumption checks.

At the end of this notebook we are create a final analytic dataset for further analysis, visualizations and modeling. Once confirmed, this notebook will be convered into a script. 

In [ ]:
#Packages import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Dataset Introduction

This `initial_analytic_dataset.csv` holds all the columns of interest from the first round of data cleaning. The data includes high level information starting with population data for all county areas, then broadband connection percent, households median income, unemployment rates, and ends with information about devices in each household. These variables were chosen because factors, such as income ane unemployment, may have a relationship or impact whether or not households have access to technology or internet. 

Later in the notebook, other columns will be created with addtional datasets, such as region and division, to visualize factors in subsections of the country. 

In [ ]:
file = open("initial_analytic_dataset.csv", "r")
analytic_data = pd.read_csv("initial_analytic_dataset.csv")

In [ ]:
for col in analytic_data.columns:
    print(col)

Note from Millie: @JJ - Can you provide documentation onw hat the different "est_total_households_with_no" are? I am assuming that it mean no for a specific device, perhaps the one above, but I would like that confirmed.

In [ ]:
# Reformating broadband_pct
analytic_data["broadband_pct"] = analytic_data["broadband_pct"] *100
analytic_data["broadband_pct"]

---

## EDA Exploration

Under this section, we are graphing the relationship between variables overall. Later on region is added to the dataset to look at the relationsips in smaller subsections to gather more insights. 

#### Broadband Percent v. Percent Total Population Enrolled

In [ ]:
analytic_data["broadband_pct"].plot.hist(figsize=(15, 5))
plt.title("Broadband Percentage Distribution")
plt.xlabel("Broadband Percentage")

In [ ]:
analytic_data["percent_total_pop_enrolled"].plot.hist(figsize=(15, 5))
plt.title("Total Population Enrolled")
plt.xlabel("Total Pop Enrolled")

In [ ]:
plt.scatter(x = analytic_data["broadband_pct"], y = analytic_data["percent_total_pop_enrolled"]) # The plot
plt.title("Population Enrolled vs Broadband Percentage") # Adding a title
plt.xlabel("Broadband Percentage") # Adding axis labels
plt.ylabel("Total Population Enrolled")

In [ ]:
#Note from Millie: I added this plot because I thought it might be more useful than looking at three different plots
sns.jointplot(data = analytic_data, x = "broadband_pct", y = "percent_total_pop_enrolled", height = 8); 

In [ ]:
#### Broadband Percent vs Unemployment

In [ ]:
sns.jointplot(data = analytic_data, x = "broadband_pct", y = "est_unemp_pop_ratio_16_over", 
              kind = "hex", height = 8); 

In [ ]:
sns.jointplot(data = analytic_data, x = "broadband_pct", y = "est_unemp_pop_ratio_16_over", 
              kind = "reg", height = 8); 

### Distrubution of Devices in Households

For now, looking at all households with some sort of device against total households.

In [ ]:
#selecting certian variables
devices_in_hh = analytic_data[['id','est_total_households_frm_devices',
'est_total_households_with_device',
'est_total_households_with_desktop',
'est_total_households_with_smartphone',
'est_total_households_with_portable',
'est_total_households_with_other',
'est_total_households_none']]

In [ ]:
#choosing columns to pivot
to_pivot = devices_in_hh[[
'est_total_households_with_device',
'est_total_households_with_desktop',
'est_total_households_with_smartphone',
'est_total_households_with_portable']]

#reshaping data
plot_devices_df = pd.melt(devices_in_hh, id_vars = 'id', value_vars = to_pivot, var_name = "type", value_name = "value")

#creating a separate value column to place decimal in different place
plot_devices_df['value2'] = plot_devices_df['value']*.001

In [ ]:
plot_devices_df.describe()

There is a huge std and data is heavily skewed to the right. 

In [ ]:
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(plot_devices_df, row="type", aspect=9, height=1.2, palette = pal)

g.map_dataframe(sns.kdeplot, x="value2", fill=True, alpha=1)
g.map_dataframe(sns.kdeplot, x="value2", color='black')
g.fig.subplots_adjust(hspace=-.5)


g.set_titles("")
g.set(yticks=[])
g.despine(left=True)

#### Employment Data Distribution

In [ ]:
employment_info = analytic_data[['id', 'est_total_pop_16_over', 'est_emp_pop_ratio_16_over', 
                                 'est_unemp_pop_ratio_16_over']]

plot_employ_df = pd.melt(employment_info, id_vars = 'id', value_vars = ['est_emp_pop_ratio_16_over', 
                                 'est_unemp_pop_ratio_16_over'], var_name = "type", value_name = "value")

plot_employ_df['value2'] = plot_devices_df['value']*.001

plot_employ_df.describe()


In [ ]:
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(plot_employ_df, row="type", aspect=9, height=1.2, palette = pal)

g.map_dataframe(sns.kdeplot, x="value", fill=True, alpha=1)
g.map_dataframe(sns.kdeplot, x="value", color='black')
g.fig.subplots_adjust(hspace=-.5)


g.set_titles("")
g.set(yticks=[])
g.despine(left=True)

Density plot above is incomplete. Currently troubleshooting. 

References for facet density plot:
- https://seaborn.pydata.org/examples/kde_ridgeplot.html
- https://python.plainenglish.io/ridge-plots-with-pythons-seaborn-4de5725881af

---

## Adding Region Dimension to the Analytic Data

This is to make it easier to breakdown the data visuals so we are not looking at the entire country on one level. All visuals below are exploring with region column. 

In [ ]:
regions = pd.read_csv("https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv")
regions.columns = regions.columns.str.lower().str.replace(" ", "_")
regions

In [ ]:
revised_analytic_data = analytic_data.merge(regions, on = "state")

In [ ]:
sns.jointplot(data = revised_analytic_data, x = "broadband_pct", y = "est_unemp_pop_ratio_16_over", 
             hue = "region", height = 8); 

### Broadband Percent by Region

In [ ]:
#broadband percent
bb_region = revised_analytic_data[['id', 'region', 'broadband_pct']]
sns.displot(
    bb_region, x="broadband_pct", col="region",
     height=3, facet_kws=dict(margin_titles=True),
);

Viusally speaking, there seems to be a differnce by region when looking at the broadband percentage with the South trailing behind the rest of the regions.

### Unemployment Ratio by Region

In [ ]:
#umemployment ratio
sns.displot(
    revised_analytic_data, x="est_unemp_pop_ratio_16_over", col="region",
    height=3, facet_kws=dict(margin_titles=True)
);

Visually, the unemployment rate seens to around 5% by reion, except for the midwest where it appears to be lower. 

### Median Income by Region

In [ ]:
#creating a separate value column to place decimal in different place
revised_analytic_data['x_est_med_income_households'] = revised_analytic_data['est_med_income_households']*.001

#median income
sns.displot(
    revised_analytic_data, x="x_est_med_income_households", col="region",
    height=3, facet_kws=dict(margin_titles=True),
);


### Analyzing Population Classification
Many Rural and Micro Areas do not have Census data for most of our data. Therefore, for smaller, more rural areas, we can only look at broadband data. We may consider breaking down the Metro Areas into different bins.

In [ ]:
#creating a separate value column to place decimal in different place
revised_analytic_data['broadband_pct'] = revised_analytic_data['broadband_pct']

#median income
sns.displot(
    revised_analytic_data, x="broadband_pct", col="pop_class",
    height=3, facet_kws=dict(margin_titles=True),
);


------

## Adding Internet Price Data

- Internet prices are on the state level, therefore all counties in the state will get the same price
- Each column on price is in US dollars as the cost per mbps
- Columns ending in a number is the total price for the for X mbps package
    - 25 is the lowest recommended amount for any household to be functional (low end)
    - 100 is the lowest recommended amount for a household of 4 (optimal low end)

In [ ]:
internet_price = pd.read_csv("Initial Clean Data/internet_price_data.csv")
revised_analytic_data = revised_analytic_data.merge(internet_price, on = "state", how = "outer")

---

## Creating Scores

#### Notes and Reminders

We are scoring variables to have an easier way to show the levels of need in the counties. At this point in the code. The higher the score the lesser the need (we might have to reverse that). 

### Income

There are various definition of poverty level or income thresholds depending on the size of the household and location. This doucmentation on poverty level for 2018 give thresholds for up to 8 persons in a household: https://aspe.hhs.gov/topics/poverty-economic-mobility/poverty-guidelines/prior-hhs-poverty-guidelines-federal-register-references/2018-poverty-guidelines

There is additional documentation households data here:https://www.census.gov/quickfacts/fact/note/US/HSD410219

With this new information, instead of choosing an arbitruary number of persons in a household and blindly applying it to all, we will need to add infomration from the households and families dataset. 

Note: Millie is creating a new notebook to view and test the data cleaning and then merging it into the initial clean dataset. 

For simplicity, we are working with the median income for households only (we can include families later if we are interested in that feature). We are applying the highest poverty level to all counties. 

In [ ]:
#Observing income information 
income_info = analytic_data[['id','est_total_households_official', 'est_avg_household_size',
                             'est_med_income_households',
                             'est_total_families_official','est_avg_families_size',
                             'est_med_income_families']]

income_info["rnd_avg_household_size"] = income_info.loc[:,('est_avg_household_size')].round(0)
income_info["rnd_avg_families_size"] = income_info.loc[:, ('est_avg_families_size')].round(0)
#trying to get rid of the "setting with copy" warning


In [ ]:
income_info["mod_est_med_income_households"] = income_info["est_med_income_households"]*.001

plt.figure(figsize=(15, 5))
sns.histplot(data=income_info,
             x="mod_est_med_income_households",
             kde=True)
plt.xlabel("Thousands of U.S. Dollars")
plt.title("Majority of families in U.S. counties have incomes around $60K"); 

In [ ]:
income_info["rnd_avg_household_size"].unique() #highest avg household is 4

In [ ]:
income_info["rnd_avg_families_size"].unique() # highest family household is 5

In [ ]:
#Creating text label for income 
def income_label(est_med_income_households):
    """Assign labels to med county income and household size
    """
    if est_med_income_households < 31380: # highest poverty line is household of 4 in Alaska
        label = "Below Poverty Line"
    elif est_med_income_households >= 31380 and est_med_income_households < 50000: 
        label = "Just Above Poverty"
    elif est_med_income_households >= 50000 and est_med_income_households < 70000: 
        label = "Middle Income"
    elif est_med_income_households >= 70000 and est_med_income_households < 90000: 
        label = "Upper Middle Income"
    elif est_med_income_households >= 90000 and est_med_income_households != 'nan': 
        label = "Upper Income"
    else: #Greater than or equal to 90k
        label = "Unknown"
    return label

#Creating the dict for numeric score of income 
income_score_dict = dict({'Below Poverty Line': 1, 
                         'Just Above Poverty': 2, 
                         'Middle Income': 3, 
                         'Upper Middle Income': 4,
                         'Upper Income': 5})

#applying the labels
revised_analytic_data["income_label"] = revised_analytic_data["est_med_income_households"].apply(income_label)

#applying the score
revised_analytic_data["income_score"] = revised_analytic_data["income_label"].map(income_score_dict)

As you may notice the income score has fewer non-null values than income_label and that is because a score was not applied if the label == "Unknown"

In [ ]:
check1 = revised_analytic_data[["id", "income_label", "income_score"]]

check1 = check1.groupby(["income_label", "income_score"]).count()

check1

## Device Score (by each device)

### Desktop Percent Score

NOTE FROM JJ: Nan values are being considered as 0 let me know if this is what we want to do and I can adjust.

In [ ]:
def pct_desktop_rank(pct_total_households_with_desktop):
    """Assign labels to county according to their broadband pct
    """
    if pct_total_households_with_desktop >= 50.963 and pct_total_households_with_desktop < 59.495: ##Less than 2 standard deviations below
        label = 1
    elif pct_total_households_with_desktop < 68.027: ##Less than 1 standard deviations below
        label = 2
    elif pct_total_households_with_desktop < 85.091: ##Less than 1 standard deviations above
        label = 3
    elif pct_total_households_with_desktop < 93.623: ##Less than 2 standard deviations above
        label = 4
    elif pct_total_households_with_desktop > 93.623 and pct_total_households_with_desktop < 105.155: ##Greater than 2 standard deviations above
        label = 5   
    else: 
        label = float("nan")
    return label
revised_analytic_data["desktop_score"] = revised_analytic_data["pct_total_households_with_desktop"].apply(pct_desktop_rank)


In [ ]:
revised_analytic_data["desktop_score"].value_counts()

### Smartphone Score

In [ ]:
def pct_smartphone_rank(pct_total_households_with_smartphone):
    """Assign labels to county according to their broadband pct
    """
    if pct_total_households_with_smartphone >= 66.27 and pct_total_households_with_smartphone < 71.983 : ##Less than 2 standard deviations below
        label = 1
    elif pct_total_households_with_smartphone < 77.696: ##Less than 1 standard deviations below
        label = 2
    elif pct_total_households_with_smartphone < 89.122: ##Less than 1 standard deviations above
        label = 3
    elif pct_total_households_with_smartphone < 94.835: ##Less than 2 standard deviations above
        label = 4
    elif pct_total_households_with_smartphone > 94.835 and pct_total_households_with_smartphone < 100.548: ##Greater than 2 standard deviations above
        label = 5  
    else: 
        label = float("nan")
    return label
revised_analytic_data["smartphone_score"] = revised_analytic_data["pct_total_households_with_smartphone"].apply(pct_smartphone_rank)



In [ ]:
revised_analytic_data["smartphone_score"].value_counts()

### Portable Device Score

In [ ]:
def pct_portable_rank(pct_total_households_with_portable):
    """Assign labels to county according to their broadband pct
    """
    if pct_total_households_with_portable >= 36.78 and pct_total_households_with_portable < 45.083: ##Less than 2 standard deviations below
        label = 1
    elif pct_total_households_with_portable <  53.386: ##Less than 1 standard deviations below
        label = 2
    elif pct_total_households_with_portable < 69.992: ##Less than 1 standard deviations above
        label = 3
    elif pct_total_households_with_portable < 78.295: ##Less than 2 standard deviations above
        label = 4
    elif pct_total_households_with_portable > 78.295 and pct_total_households_with_portable < 86.598: ##Greater than 2 standard deviations above
        label = 5   
    else: 
        label = float("nan")
    return label
revised_analytic_data["portable_score"] = revised_analytic_data["pct_total_households_with_portable"].apply(pct_portable_rank)


In [ ]:
revised_analytic_data["portable_score"].value_counts()

### No Device Score

In [ ]:
def pct_no_device_rank(pct_total_households_no_device):
    """Assign labels to county according to their broadband pct
    """
    if pct_total_households_no_device  >= -(4.192) and pct_total_households_no_device < 0.096: ##Less than 2 standard deviations below
        label = 1
    elif pct_total_households_no_device < 4.384: ##Less than 1 standard deviations below
        label = 2
    elif pct_total_households_no_device < 12.96: ##Less than 1 standard deviations above
        label = 3
    elif pct_total_households_no_device < 17.248: ##Less than 2 standard deviations above
        label = 4
    elif pct_total_households_no_device > 17.248 and pct_total_households_no_device < 21.536: ##Greater than 2 standard deviations above
        label = 5   
    else: 
        label = float("nan")
    return label
revised_analytic_data["no_device_score"] = revised_analytic_data["pct_total_households_no_device"].apply(pct_no_device_rank)


In [ ]:
revised_analytic_data["no_device_score"].value_counts()

## Broadband Score

Broadband Percentage Score was assigned using percentage ranges defined by Census Bureau Data Article found at https://www.census.gov/library/stories/2018/12/rural-and-lower-income-counties-lag-nation-internet-subscription.html. Scores were assigned with 5 being the lowest percentage and 1 being the highest percentage. 

In [ ]:
revised_analytic_data["broadband_pct"].describe()

In [ ]:
#creating broadband classification
def bdbd_type(broadband_pct):
    """Assign labels to county according to their broadband pct
    """
    if broadband_pct < 54.9 : #Between 0.0 to 54.9
        label = 1
    elif broadband_pct < 64.9: #Between 55.0 to 64.9
        label = 2
    elif broadband_pct < 74.9: #Between 65.0 to 74.9
        label = 3
    elif broadband_pct < 84.9: #Between 75.0 to 84.9
        label = 4
    elif broadband_pct > 85.0: #Greater than 85.0
        label = 5  
    else: 
        label = 1
    return label

In [ ]:
revised_analytic_data["bdbd_score"] = revised_analytic_data["broadband_pct"].apply(bdbd_type)

In [ ]:
revised_analytic_data["bdbd_score"].value_counts()

In [ ]:
#Selecting only columns of interest
review5 = revised_analytic_data[["id", "bdbd_score"]]

#grouping by newly created "portable_rank" and counting 
review5 = review5.groupby("bdbd_score").count()

review5

In [ ]:
bdbd_df = revised_analytic_data[["county", "state","pop_total", "pop_class", "broadband_pct", "bdbd_score"]]
bdbd_df

In [ ]:
corrz = bdbd_df.corr()
corrz

In [ ]:
my_contingency_table = pd.crosstab(index=bdbd_df["pop_class"], columns=bdbd_df["bdbd_score"])
my_contingency_table

In [ ]:
sns.heatmap(my_contingency_table)

In [ ]:
sns.heatmap(my_contingency_table, cmap="Reds")

In [ ]:
norm_con = pd.crosstab(index=bdbd_df["pop_class"], columns=bdbd_df["bdbd_score"], normalize="columns")*100
sns.heatmap(norm_con)

In [ ]:
revised_analytic_data.info()

In [ ]:
#Selecting columns with the scores 
cols_to_sum = ["income_score", "desktop_score", "smartphone_score", 
            "portable_score", "no_device_score", "bdbd_score"]

# adding up all the scores created for a "total score"
revised_analytic_data["total_score"] = revised_analytic_data[cols_to_sum].sum(axis=1)

> NOTE: When the times comes with correct devices data scoring is incorporated, this can be regenerated and named as final.

In [ ]:
#Exporting csv file
revised_analytic_data.to_csv(r'revised_analytic_dataset.csv', index = False)